# Problem 1 : RAG

In [2]:
from beyondllm import source, llms, retrieve, generator
from beyondllm.embeddings import FineTuneEmbeddings

In [3]:
from getpass import getpass
import os

google_api_key = getpass('Enter Your Google API Key')
os.environ['GOOGLE_API_KEY'] = google_api_key

Enter Your Google API Key ········


### Load Data

In [4]:
path = "DS-Alvin Rachmat 28 Aug 24 v4.32.pdf"
data = source.fit(path, dtype="pdf", chunk_size=2000, chunk_overlap=2)

### Training Part

Here we will use Gemini model, with the embedding model of BAAI small en v.15.

In [4]:
llm = llms.GeminiModel()

In [5]:
fine_tuned_model = FineTuneEmbeddings()

In [6]:
embed_model = fine_tuned_model.train([path], "BAAI/bge-small-en-v1.5", llm, "fintune")

Parsing nodes: 100%|███████████████| 2/2 [00:00<00:00, 1025.13it/s]


Parsed 2 nodes


Parsing nodes: 100%|████████████████| 1/1 [00:00<00:00, 993.44it/s]


Parsed 1 nodes


100%|████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


...


Iteration: 100%|█████████████████████| 1/1 [00:00<00:00,  2.69it/s]

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Information Retrieval Evaluation of the model on the  dataset after epoch 1:
Information Retrieval Evaluation of the model on the  dataset after epoch 1:
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Queries: 4
Queries: 4
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Corpus: 1

Corpus: 1

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Score-Function: cos_sim
Score-Function: cos_sim
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@1: 100.00%
Accuracy@1: 100.00%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@3: 100.00%
Accuracy@3: 100.00%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@5: 100.00%
Accuracy@5: 100.00%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@10: 100.00%
Accuracy@10: 100.00%
INFO:sentence_transformers.evaluation.Informatio


Epoch: 100%|█████████████████████████| 2/2 [00:02<00:00,  1.02s/it]

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: fintune
Load pretrained SentenceTransformer: fintune


INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


### Load Model

In [7]:
embed_model = fine_tuned_model.load_model("fintune")
retriever = retrieve.auto_retriever(data, embed_model, type="normal", top_k=6)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: fintune
Load pretrained SentenceTransformer: fintune
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


### Test a Question

Here we can ask anything related to CV. If it doesn't exist in CV, then the model will throw something like, not given in context.

In [9]:
pipeline = generator.Generate(question="Explain any alvin's cloud related work",retriever=retriever,llm=llm)

Batches: 100%|██████████████████████| 1/1 [00:00<00:00, 163.45it/s]


In [10]:
print(pipeline.call())

Alvin has experience with Google Cloud, Microsoft Azure, and FastAPI. He has used Google Cloud for Compute Engine, Cloud Run, Cloud Storage, Logging Sink, Pub/Sub, Cloud Build and Trigger, Cloud Artifact, Vertex, and more. He has also used Microsoft Azure and FastAPI for various projects.


### Evaluating The RAG Model

In this part we can evaluate a model based on the context relevancy, answer relevancy and groundness score. 

In [11]:
print(pipeline.get_rag_triad_evals())

Executing RAG Triad Evaluations...
Context relevancy Score: 1.0
This response does not meet the evaluation threshold. Consider refining the structure and content for better clarity and effectiveness.
Answer relevancy Score: 8.0
This response meets the evaluation threshold. It demonstrates strong comprehension and coherence.
Groundness score: 7.0
This response does not meet the evaluation threshold. Consider refining the structure and content for better clarity and effectiveness.


The streamlit can be accessed here, on a ngrok deployment.  
https://prime-utterly-glowworm.ngrok-free.app/

# Problem 2 : Agent

Import the needed library. Here dont forget to input your groq token  
https://console.groq.com/

I put lower temperature. Higher temperature causes the agent to be really strict about the question and asking in very detail manner causing endless question.

In [42]:
from openagi.llms.groq import GroqModel
from openagi.planner.task_decomposer import TaskPlanner
from openagi.actions.tools.ddg_search import DuckDuckGoSearch
from openagi.agent import Admin
from openagi.memory import Memory
import os
from getpass import getpass

os.environ['GROQ_API_KEY'] = getpass('Enter Your Google API Key')
os.environ['GROQ_MODEL'] = "llama-3.1-8b-instant"
os.environ['GROQ_TEMP'] = "0.3"

Enter Your Google API Key ········


In [36]:
config = GroqModel.load_from_env_config()
llm = GroqModel(config=config)

Here we dfine human intervene to be true, means agent can interact later with us.

In [37]:
planner = TaskPlanner(human_intervene=True)
action = DuckDuckGoSearch

In [38]:
admin = Admin(
    actions= [action],
    planner = planner,
    memory=Memory(),
    llm = llm,
    max_iterations = 10
)

2024-08-28 22:35:09 :from_kwargs: Using Chroma persistent client with path: /tmp/openagi
2024-08-28 22:35:09 :__init__: Session ID initialized: 7f6482589cf94f17bceb0859a80c6d7e


Here i ask simple recipe for omelette, including the interactions.

In [39]:
res = admin.run(
    query="Write simple recipe about omelette!",
    description="You are chef who know all of the food recipe in the world. Just be simple.",
)

2024-08-28 22:35:12 :run: Running Admin Agent...
2024-08-28 22:35:12 :run: SessionID - 7f6482589cf94f17bceb0859a80c6d7e
2024-08-28 22:35:12 :run_planner: Thinking...
2024-08-28 22:35:12 :human_clarification: Initiating Human Clarification. Make sure to clarify the questions, if not just type `I dont know` to stop
2024-08-28 22:35:12 :_send_single_request: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Agent: Would you like the omelette recipe to be vegetarian, non-vegetarian, or can it include any ingredients?
You:  vegan


2024-08-28 22:35:18 :_send_single_request: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Agent: What specific type of vegan omelette are you looking for (e.g., tofu-based, chickpea-based, mushroom-based)?
You:  tofu-based


2024-08-28 22:35:27 :_send_single_request: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


...

2024-08-28 22:36:12 :_send_single_request: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Agent: You've specified the omelette as 'mildly spicy', but you've also mentioned it should be 'spicy enough'. Can you confirm if you want an omelette that is mildly spicy but still meets the 'spicy enough' criteria, or if you're looking for a different level of spiciness?
You:  I don't know


2024-08-28 22:36:16 :get_prompt: Using prompt: SingleAgentTaskCreator
2024-08-28 22:36:18 :_send_single_request: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-08-28 22:36:18 :run: Tasks Planned...
2024-08-28 22:36:18 :save_task: Task saved: 9190ed25592f40708aa46f8b56a2560a
2024-08-28 22:36:18 :save_task: Task saved: cb232e9309ad47e8a0d093f5417f955c
2024-08-28 22:36:18 :save_task: Task saved: f8810affbb734eb89dbaec3d7c5c4c1a




Tasks: [{'task_name': 'Define the type of omelette', 'description': 'Use DuckDuckGoSearch to find the definition of a vegan tofu-based omelette.'}, {'task_name': 'Determine the level of spiciness', 'description': 'Use DuckDuckGoSearch to find the definition of mildly spicy and firm firm tofu.'}, {'task_name': 'Find a suitable recipe', 'description': 'Use DuckDuckGoSearch to find a simple recipe for a vegan tofu-based omelette that is mildly spicy and firm firm tofu.'}, {'task_name': 'Extract ingredients from the recipe', 'description': 'Use MemoryRagAction to extract the ingredients from the recipe found in the previous task.'}, {'task_name': 'Extract instructions from the recipe', 'description': 'Use MemoryRagAction to extract the instructions from the recipe found in the previous task.'}, {'task_name': 'Extract cooking time and temperature from the recipe', 'description': 'Use MemoryRagAction to extract the cooking time and temperature from the recipe found in the previous task.'},

2024-08-28 22:36:18 :save_task: Task saved: f82f034b47964524bcde922ed6027833
2024-08-28 22:36:18 :save_task: Task saved: 1c7d7bdff3fd44e19d7b64d2f4b1d659
2024-08-28 22:36:18 :save_task: Task saved: fdc95e749850485b853b372d4a16aaeb
2024-08-28 22:36:18 :save_task: Task saved: d163e84443854df18d3e3f6fa2ce162f
2024-08-28 22:36:18 :single_agent_execution: **** Executing Task - Define the type of omelette [9190ed25592f40708aa46f8b56a2560a] ****
2024-08-28 22:36:19 :_send_single_request: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-08-28 22:36:19 :single_agent_execution: LLM execution completed. Observations: Begin!
Observation: None
Thought:
To answer the question, "Define the type of omelette. Use DuckDuckGoSearch to find the definition of a vegan tofu-based omelette," I need to think about how to approach this problem. 

First, I need to consider what a vegan tofu-based omelette is. It's a type of omelette that is made with tofu instead of eggs,

Here is the result from agent.

In [40]:
print(res)

**Vegan Tofu-Based Omelette Recipe**

### Final Output
To make a simple vegan tofu-based omelette, you will need the following ingredients: tofu, spices, and vegetables. Cooking time is 10-15 minutes and temperature is 375°F.

### Ingredients and Instructions
#### Ingredients Needed
- Tofu
- Spices
- Vegetables

#### Cooking Instructions
1. Crumbling tofu into small pieces
2. Mixing spices and vegetables
3. Cooking tofu and spices mixture in a pan
4. Folding the omelette in half and serving
